In [34]:
%load_ext autoreload

In [35]:
from setup_imports import *  # noqa: F401,F403

✅ Added c:\Users\andyl\Python\audio-language-trainer to Python path
✅ 'src' module is now importable


In [45]:
%autoreload 2
from src.phrases.phrase_model import get_firestore_client, Phrase

In [46]:
p = Phrase.create_phrase('Hello World!')

In [47]:
p.model_dump()

{'phrase_hash': 'hello_world_7f83b1',
 'english': 'Hello World!',
 'english_lower': 'hello world!',
 'tokens': ['world', 'hello'],
 'verbs': [],
 'vocab': ['world', 'hello'],
 'source': 'generated',
 'translations': [{'phrase_hash': 'hello_world_7f83b1',
   'language': 'en-GB',
   'text': 'Hello World!',
   'text_lower': 'hello world!',
   'tokens': ['world', 'hello'],
   'audio': []}]}

In [13]:
phrase = Phrase(
            english="she runs to the store daily?",
            english_lower="she runs to the store daily",
            tokens=["she", "runs", "to", "the", "store", "daily"],
            lemmas=["she", "run", "to", "the", "store", "daily"],
            verbs=["run"],
            vocab=["she", "to", "the", "store", "daily"],
            source="manual"
        )

In [14]:
phrase.get_phrase_hash()

'she_runs_to_the_store_daily_494b6e'

In [1]:
import langcodes

In [ ]:
def validate_language_tag(tag: str) -> str:
    """Validate and standardize a language tag using langcodes."""

    language = langcodes.get(tag)
    if language.is_valid() and language.territory:
        return language.to_tag()
    else:
        raise ValueError(f"Invalid language tag: {tag}")


In [33]:
langcodes.Language.get('en-UK').language_name()

'English'

In [5]:
langcodes.standardize_tag('en')

'en'

In [3]:
import langcodes

# Validate a language code
code = langcodes.Language.get('fr-FR')
print(code.is_valid())  # True

# Normalize codes
print(langcodes.standardize_tag('en_US'))  # 'en-US'
print(langcodes.standardize_tag('zh-hans-cn'))  # 'zh-Hans-CN'

# Get human-readable names
lang = langcodes.Language.get('fr-FR')
print(lang.display_name())  # 'French'
print(lang.display_name('fr'))  # 'français'
print(lang.describe('en'))  # {'language': 'French', 'region': 'France'}

# Parse components
lang = langcodes.Language.get('zh-Hans-CN')
print(lang.language)  # 'zh'
print(lang.script)  # 'Hans'
print(lang.region)  # 'CN'

# Check if code exists/is valid
try:
    lang = langcodes.Language.get('zz-ZZ')
except langcodes.LanguageTagError:
    print("Invalid code")

# Get all language names in a specific language
lang = langcodes.Language.get('sv-SE')
print(lang.autonym())  # 'svenska' (language's own name)
print(lang.display_name('en'))  # 'Swedish'


True
en-US
zh-Hans-CN
French (France)
français (France)
{'language': 'French', 'territory': 'France'}
zh
Hans
CN
svenska (Sverige)
Swedish (Sweden)
